In [ ]:
import pandas as pd
import asyncio
import kagglehub

import openai
from openai import OpenAI
import nest_asyncio
import aiohttp

nest_asyncio.apply()

RANDOM_STATE = 123

EMOTIONS_MAPPING = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# These are my personal API account keys. Please, do not use them for bad pursposes(
API_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent"
OPENAI_API_KEY = "sk-proj-Rdg95JoulgrdT26zS-5t-lc7lBEUwbw9lzzaHAZgBJfmIie5CouR2T3BlbkFJfrd7c_zFsneNprH9nJijBqMjNBNj29m28O0ijWwFpXjinEJVZBYDUB_9IkJG39TQrfnemtbCoA"
GEMINI_API_KEY = "AIzaSyBSZSvyvy6ITFgVt"

In [2]:
prompt1 = (
    f"Classify the emotion of the given text into one of these categories: "
    f"{', '.join(EMOTIONS_MAPPING.values())}.\n\n"
    "Text: \"{}\"\n\n"
    f"Response must be a single word (in lower case) - the emotion from given list - without any additional text:"
)

PROMPT = prompt1

In [3]:
async def detect_emotion_openai(text):
    try:
        client = openai.AsyncOpenAI(api_key=OPENAI_API_KEY)
        completion = await client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {
                    "role": "user",
                    "content": PROMPT.format(text)
                }
            ]
        )
        
        await asyncio.sleep(0.5) 
        return completion.choices[0].message.content.strip()

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [4]:
async def process_emotions(X, y, count, detect_function):
    tasks = []
    max_count = min(len(X), count)
    failed_y = []

    for i, (text, emotion) in enumerate(zip(X, y)):
        if i >= max_count:
            break
        tasks.append(detect_function(text))

    results = await asyncio.gather(*tasks)

    successes = 0
    for i, (text, emotion, model_res) in enumerate(zip(X, y, results)):
        if i >= max_count:
            break
        if model_res and model_res == emotion:
            successes += 1
        else:
            failed_y.append({
                "text": text,
                "real_emotion": emotion,
                "predicted_emotion": model_res
            })

    return successes, failed_y

In [13]:
path = kagglehub.dataset_download("aadyasingh55/twitter-emotion-classification-dataset")
file_path = path + '/train-00000-of-00001.parquet'
df = pd.read_parquet(file_path)
df = df.sample(100, random_state=RANDOM_STATE)

X = df['text']
y = df['label'].map(EMOTIONS_MAPPING)

In [14]:
async def openai_emotions():
    successes, failed_y = await process_emotions(X, y, 100, detect_emotion_openai)
    return successes, failed_y
openai_results = asyncio.run(openai_emotions())

In [15]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)

generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 50,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

In [16]:
async def detect_emotion_gemini_single(text):
    try:
        chat_session = model.start_chat(history=[])
        response = await asyncio.to_thread(chat_session.send_message, PROMPT.format(text))
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred with Gemini: {e}")
        return None

In [17]:
async def detect_emotion_gemini(X, y, interval=1):
    results = []
    for text, emotion in zip(X, y):
        try:
            result = await detect_emotion_gemini_single(text)
            results.append({"text": text, "real_emotion": emotion, "predicted_emotion": result})
        except Exception as e:
            print(f"Error processing text: {text} - {e}")
        await asyncio.sleep(interval)
    return results

In [18]:
async def gemini_emotions(X, y, interval):
    results = await detect_emotion_gemini(X, y, interval)
    successes = sum(1 for result in results if result["predicted_emotion"] == result["real_emotion"])
    failed_y = [result for result in results if result["predicted_emotion"] != result["real_emotion"]]
    return successes, failed_y

In [23]:
gemini_results = await gemini_emotions(X, y, interval=3)

An error occurred with Gemini: 429 Resource has been exhausted (e.g. check quota).
An error occurred with Gemini: 429 Resource has been exhausted (e.g. check quota).
An error occurred with Gemini: 429 Resource has been exhausted (e.g. check quota).
An error occurred with Gemini: 429 Resource has been exhausted (e.g. check quota).
An error occurred with Gemini: 429 Resource has been exhausted (e.g. check quota).


In [24]:
print("OpenAI Results:")
print(f"Successes: {openai_results[0]}")
print(f"Failures: {openai_results[1]}")

print("\nGemini Results:")
print(f"Successes: {gemini_results[0]}")
print(f"Failures: {gemini_results[1]}")

OpenAI Results:
Successes: 65
Failures: [{'text': 'i miss having someone to talk to who i have that feeling about and who is pleasant and lovely', 'real_emotion': 'joy', 'predicted_emotion': 'love'}, {'text': 'i just feel like i get blamed for everything', 'real_emotion': 'sadness', 'predicted_emotion': 'anger'}, {'text': 'i also apologize for mentioning about him in my previous post i don t know if he would feel offended if he ever reads my blog but i didn t mean it so much that way afterall if a relationship ends both parties would have their faults', 'real_emotion': 'anger', 'predicted_emotion': 'sadness'}, {'text': 'im finding is the difference in having a life that leaves you wanting and one that feels truly rich', 'real_emotion': 'joy', 'predicted_emotion': 'sadness'}, {'text': 'i feel like nothing ever gets resolved because there is never enough time in the day to talk', 'real_emotion': 'joy', 'predicted_emotion': 'frustration'}, {'text': 'i feel i have missed some key points no

In [42]:
openai_results[0]

65

In [43]:
gemini_results[0]

55

In [41]:
openai_texts = {item['text'] for item in openai_results[1]}
gemini_texts = {item['text'] for item in gemini_results[1]}

common_texts = openai_texts & gemini_texts

common_results = []
for item in openai_results[1]:
    if item['text'] in common_texts:
        gemini_prediction = next((g['predicted_emotion'] for g in gemini_results[1] if g['text'] == item['text']), None)
        common_results.append({
            'text': item['text'],
            'real_emotion': item['real_emotion'],
            'openai_predicted_emotion': item['predicted_emotion'],
            'gemini_predicted_emotion': gemini_prediction
        })

for result in common_results:
    print(f"Text: {result['text']}")
    print(f"Real Emotion: {result['real_emotion']}")
    print(f"OpenAI Predicted Emotion: {result['openai_predicted_emotion']}")
    print(f"Gemini Predicted Emotion: {result['gemini_predicted_emotion']}")
    print("---")

Text: i miss having someone to talk to who i have that feeling about and who is pleasant and lovely
Real Emotion: joy
OpenAI Predicted Emotion: love
Gemini Predicted Emotion: sadness
---
Text: i just feel like i get blamed for everything
Real Emotion: sadness
OpenAI Predicted Emotion: anger
Gemini Predicted Emotion: anger
---
Text: i also apologize for mentioning about him in my previous post i don t know if he would feel offended if he ever reads my blog but i didn t mean it so much that way afterall if a relationship ends both parties would have their faults
Real Emotion: anger
OpenAI Predicted Emotion: sadness
Gemini Predicted Emotion: sadness
---
Text: im finding is the difference in having a life that leaves you wanting and one that feels truly rich
Real Emotion: joy
OpenAI Predicted Emotion: sadness
Gemini Predicted Emotion: sadness
---
Text: i feel like nothing ever gets resolved because there is never enough time in the day to talk
Real Emotion: joy
OpenAI Predicted Emotion: fr